<a href="https://colab.research.google.com/github/JiaXiao-Peter/peter.github.io/blob/main/Schelling_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Build Your Own LLM Society**

### **Cell 1: Test the API Connection**

First, let's make sure the API works. This is the last code from our previous class, establishing the basic API connection.

In [ ]:

from openai import OpenAI
import os

# Connect to Shubiaobiao API
client = OpenAI(
    base_url="https://api.shubiaobiao.cn/v1/",
    api_key="sk-7MPDKIdGJngMDr5i5a15907dF4Ec4c8294AfB58dAe43Db5e",
)

# Test a simple conversation
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Introduce yourself in one sentence"}
    ]
)

print("✅ API connected successfully!")
print("Assistant reply:", completion.choices[0].message.content)


***Explanation:***
We reuse the exact same API-setup cell from the previous lesson. Once it runs without errors, we are ready to embed LLM calls into the Schelling simulation.

### **Cell 2: Import the modular Schelling framework**

We directly reuse the modular scaffold built in Lab 2, which already decomposes the Schelling model into plug-and-play components. This lets us swap in a LLM-based decision rule without touching the rest of the pipeline.

The same `schelling/` package from Lab 2 is reused unchanged:

- `initialization.py` – how the city is seeded  
- `agent_selector.py` – which agent gets a turn  
- `target_selector.py` – which vacant cell is shortlisted  
- `move_acceptor.py` – **this is our plug-in point**; the original greedy rule will be replaced by an LLM call inside the notebook  
- `stop_criterion.py` – when to halt  
- `utils.py` – plotting & utility helpers  

None of the .py files are modified. We simply import the existing package to show how easily an external decision layer (LLM) can be plugged in.

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

import sys, os

project_root = '/content/drive/MyDrive/Schelling-simulation-Lab5'
sys.path.append(project_root)

import types, importlib
import sys as _sys
_sys.modules['imp'] = types.SimpleNamespace(reload=importlib.reload)
# -------------------------------------------

%load_ext autoreload
%autoreload 2

print("On sys.path:", project_root in sys.path)
print("Dir listing:", os.listdir(project_root))

from schelling import *
import numpy as np
print("✅ Schelling framework imported successfully!")


### **Cell 3: Initialise Baseline**

We keep the original Lab-2 template intact and simply **swap the greedy acceptance rule** for an LLM-based decision function.  
This version still uses:

- Random initialization of agents
- Random agent & target selection
- Fixed-step stop criterion

***Note:*** the move choice is now made by prompting an LLM, giving agents **memory and reasoning** while preserving the familiar data structures and control flow.

In [ ]:
# === Parameters ===
Q = 16        # number of neighbourhoods
H = 10       # capacity of each neighbourhood
rho0 = 0.5   # initial global density
max_steps = 20  # short run for demo because LLM calls cost time
memory_window = 3  # NEW: how many recent decisions the agent remembers

# === Initialise the city ===
occupied = initialization.init_random(Q, H, rho0)
print(f"Initial state: total agents {np.sum(occupied)}, mean density {np.mean(occupied/H):.2f}")

# === Visualise initial state ===
utils.plot_density_heatmap(4, 4, occupied / H, title='Initial density heat-map', xlabel='', ylabel='')

### **Cell 4: Implement Memory Mechanism**

This is a simple memory scheme. In a full model each agent could carry its own memory; here we use one shared, time-ordered list to give the LLM a sense of recent collective experience.

In [ ]:

agent_memory = []  # global chronological memory list

# Format the latest memories into a string to be inserted into the LLM prompt.
def format_memory_for_prompt(memory_list, window_size):
    if not memory_list or window_size == 0:
        return "No historical records."

    recent = memory_list[-window_size:]
    prompt_segment = ""
    for i, rec in enumerate(recent, 1):
        prompt_segment += (f"Memory {i}: at step {rec['step']} you were in neighbourhood "
                          f"with density {rec['from_density']:.2f} (utility {rec['from_utility']:.2f}) "
                          f"to density {rec['to_density']:.2f} (utility {rec['to_utility']:.2f}), "
                          f"decision='{rec['decision']}', reason='{rec['reasoning']}'.\n")
    return prompt_segment
print("✅ Memory mechanism initialised")


### **Cell 5: Define the Core LLM Decision Function**

Here we query the LLM to decide whether to move. This is the heart of the LLM-enhanced simulation.

**Parameters:**
- `from_block`, `to_block`: indices of origin and destination neighbourhoods
- `occupied`: current occupancy counts per block
- `H`: capacity of each block
- `utility_fn`: utility function `u(density)`
- `memory_list`: historical decision memory
- `window_size`: how many recent memories to include
- `client`: OpenAI client instance

**Returns:**
- `decision` (bool): `True` if LLM advises moving, `False` otherwise
- `reasoning` (str): LLM's one-sentence justification
- `from_density`, `to_density`, `from_utility`, `to_utility`: computed values

This is the simulation's "brain". We explicitly ask the LLM for JSON and add error-handling. `temperature=0.3` keeps decisions consistent.

In [ ]:
import re

def call_llm_for_move_decision(from_block, to_block, occupied, H, utility_fn,
                               memory_list, window_size, client):

    # Compute current and prospective densities and utilities
    from_density = occupied[from_block] / H
    to_density = (occupied[to_block] + 1) / H   # density after the agent arrives
    from_utility = utility_fn(from_density)
    to_utility = utility_fn(to_density)

    # Build prompt: role + context + memory + output-format instruction
    system_prompt = ("You are an agent living in a city. "
                     "Decide whether to move based on the current environment and your memories.")

    user_prompt = f"""
You currently live in neighbourhood {from_block} (density {from_density:.2f}, utility {from_utility:.2f}).
You are considering moving to neighbourhood {to_block} (density after you arrive {to_density:.2f}, expected utility {to_utility:.2f}).

Your memory window (last {window_size} relevant decisions) is:
{format_memory_for_prompt(memory_list, window_size)}

Make a choice. You prefer higher utility but can also learn from history.
**Important**: Reply ONLY a plain JSON object, no Markdown fences, no extra text:
{{"decision": "move" or "stay", "reasoning": "brief one-sentence reason"}}
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": system_prompt},
                      {"role": "user", "content": user_prompt}],
            temperature=0.3   # lower temperature for more consistent decisions
        )

        llm_output = response.choices[0].message.content.strip()
        llm_output = re.sub(r'```json|```', '', llm_output).strip()   # delete Markdown  fence
        parsed = json.loads(llm_output)

        decision = parsed.get("decision", "stay") == "move"
        reasoning = parsed.get("reasoning", "no clear reason")

        return decision, reasoning, from_density, to_density, from_utility, to_utility

    except json.JSONDecodeError as e:
        print(f"⚠️ JSON decode failed: {e}. LLM output: {llm_output[:100]}...")
        return False, "JSON decode failed", from_density, to_density, from_utility, to_utility
    except Exception as e:
        print(f"⚠️ API call failed: {e}")
        return False, "API call failed", from_density, to_density, from_utility, to_utility

print("✅ LLM decision function defined")


### **Cell 6: Main Simulation Loop**

This cell demonstrates the retrofit process. The key change is replacing the line `if move_acceptor.accept_if_personal_utility_improves(...):` with an LLM function call and memory management, showcasing the power of modular design.


In [ ]:
# === ORIGINAL Lab 2 Baseline loop (commented out) ===

# step = 0
# while True:
#     # Stopping criterion
#     if stop_criterion.stop_after_fixed_steps(step, max_steps):
#         print(f"Stopped at step {step}")
#         break

#     # 1. Select an agent block (weighted by number of agents)
#     from_block = agent_selector.select_agent_random(occupied)

#     # 2. Select a target block (must have space)
#     to_block = target_selector.select_target_random_cell(occupied, H)

#     # 3. Check if move is accepted (Δu > 0)
#     if move_acceptor.accept_if_personal_utility_improves(occupied[from_block] / H, (occupied[to_block] + 1) / H, utils.triangle_utility):
#         occupied[from_block] -= 1
#         occupied[to_block] += 1
#     step += 1

In [ ]:
import time

# === Simulation Main Loop ===
print("Starting LLM-driven social simulation...")
initial_state_snapshot = occupied.copy()

step = 0
decision_log = []  # store every decision for later analysis

while True:
    # stopping criterion
    if stop_criterion.stop_after_fixed_steps(step, max_steps):
        print(f"\n✅ LLM simulation stopped at step {step}")
        break

    # 1. select an agent
    from_block = agent_selector.select_agent_random(occupied)

    # 2. select a target neighbourhood
    to_block = target_selector.select_target_random_cell(occupied, H)

    # 3. LLM decision: call the LLM to decide whether to move (core change)
    #    we replace the simple utility comparison with LLM reasoning
    decision, reasoning, from_d, to_d, from_u, to_u = call_llm_for_move_decision(
        from_block, to_block, occupied, H, utils.triangle_utility,
        agent_memory, memory_window, client
    )

    # if LLM decides to move, update state
    if decision:
        occupied[from_block] -= 1
        occupied[to_block] += 1
        move_str = "✅MOVED"
    else:
        move_str = "❌STAYED"

    # record this decision to memory (keep window size)
    agent_memory.append({
        'step': step,
        'from_block': from_block,
        'to_block': to_block,
        'from_density': from_d,
        'to_density': to_d,
        'from_utility': from_u,
        'to_utility': to_u,
        'decision': 'move' if decision else 'stay',
        'reasoning': reasoning
    })
    if len(agent_memory) > memory_window * 10:  # cap total memory to avoid unbounded growth
        agent_memory.pop(0)

    # append to analysis log
    decision_log.append({'step': step, 'decision': decision, 'reasoning': reasoning})

    # print progress & sample decision (every 500 steps to limit output)
    print(f" step {step:5d} | Block {from_block} → {to_block} {move_str} | reason: {reasoning}")
    step += 1

print("Simulation finished!")

### **Cell 7: Visualise Final State**

In [ ]:

# Plot final-density heat map
utils.plot_density_heatmap(4, 4, occupied / H,
                           title=f'Final density heat-map', xlabel='', ylabel='')

# Compute and print some statistics
final_utilities = utils.triangle_utility(occupied / H)
print("\nSimulation summary:")
print(f"  Final mean density: {np.mean(occupied/H):.3f}")
print(f"  Final mean utility: {np.mean(final_utilities):.3f}")
print(f"  Total decisions logged: {len(decision_log)}")


### **Cell 8  Snapshot: From Initial to Final**

| What you see | Why it matters |
|--------------|----------------|
| **Density heat-maps** (top row) | Instant colour check on **where people ended up**. Brighter = more crowded. Compare left → right to spot segregation or mixing. |
| **Utility heat-maps** (bottom row) | **Individual satisfaction** in each block (0-1 scale). Greener = happier. LLM choices should push blocks toward greener shades **if** utility really guides them. |
| **Global utility** printed below | Weighted sum (∑ density × utility). Gives **one-line welfare summary** of the whole city. ↑ means LLM decisions collectively raised well-being; ↓ means they didn’t. |

Use this cell to **eyeball** whether your prompt / temperature / memory_window is producing:
- more integrated (uniform red) or segregated (hot-spots) layouts,
- higher average utility,
- or simply different global welfare.

In [ ]:

import matplotlib.pyplot as plt

# ----------  Compute statistics ----------
initial_util = utils.triangle_utility(initial_state_snapshot / H)
final_dens   = occupied
final_util   = utils.triangle_utility(final_dens / H)
delta_util   = final_util - initial_util

move_ratio   = sum(d['decision'] for d in decision_log) / len(decision_log)
mean_improve = np.nanmean(delta_util)

# ----------  Plotting ----------
rows = cols = int(np.sqrt(Q))
figsize = (cols*1.8, rows*1.8)
fig, ax = plt.subplots(2, 2, figsize=figsize, dpi=100)
fig.suptitle(f'Initial vs Final ({max_steps} steps)', fontsize=10)

def draw_heatmap(ax, data, title, cmap, vmin, vmax, fmt='.2f'):
    """helper: small heat-map with text annotation"""
    im = ax.imshow(data.reshape(rows, cols), cmap=cmap, vmin=vmin, vmax=vmax)
    ax.set_title(title, fontsize=9)
    ax.set_xticks([]); ax.set_yticks([])
    for i in range(rows):
        for j in range(cols):
            ax.text(j, i, f'{data.reshape(rows, cols)[i,j]:{fmt}}',
                    ha='center', va='center', color='black', fontsize=8)
    return im

# sub-plots
im0 = draw_heatmap(ax[0,0], initial_state_snapshot/H, 'Initial Density',  'Reds',   0, 1)
im1 = draw_heatmap(ax[0,1], final_dens/H,   'Final Density',    'Reds',   0, 1)
im2 = draw_heatmap(ax[1,0], initial_util, 'Initial Personal Utility',
                   cmap='Blues', vmin=0, vmax=1)
im3 = draw_heatmap(ax[1,1], final_util, 'Final Personal Utility',
                   cmap='Blues', vmin=0, vmax=1)

for im, axi in zip([im0, im1], ax[0,:]): plt.colorbar(im, ax=axi, shrink=0.6, pad=0.02)
for im, axi in zip([im2, im3], ax[1,:]): plt.colorbar(im, ax=axi, shrink=0.6, pad=0.02)

plt.tight_layout(pad=0.8)
plt.show()

# ----------  Global utility ----------
global_initial = np.sum(initial_state_snapshot / H * initial_util)
global_final   = np.sum(final_dens / H * final_util)
print(f"Global utility  Initial {global_initial:.3f}  →  Final {global_final:.3f}  "
      f"({'↑' if global_final > global_initial else '↓' if global_final < global_initial else '→'})")


### **🎯Core Takeaways of Lab5:**

1. **Modular Reusability**: No need to modify the original framework; introducing the LLM is achieved by replacing the decision function.
2. **Memory Mechanism**: Adds short-term memory to the agents, making their decisions historically dependent.
3. **Structured Interaction**: Uses JSON format to standardize LLM output, facilitating program parsing.
4. **Explainability**: The "reasoning" provided by the LLM makes agent behavior transparent and analyzable.

### **Suggested Explorations:**

- **Adjust Memory Window**: Change the `memory_window` to 0, 10, or 20, and observe how the decision patterns change.
- **Modify the Prompt**: Change the Prompt within `call_llm_for_move_decision`, for example:
    - Make the agent more "conservative" or more "risk-taking".
- **Collective Utility**: Modify the Prompt so that the LLM considers not only personal utility but also the impact of the move on the overall utility of both the origin and destination communities.
- **Cost Sensitivity**: Introduce the concept of "moving cost" into the Prompt and observe if the LLM consequently reduces movement.
- **Bounded Rationality**:
    - Cognitive Discounting / Search Radius: Restrict agents to only view neighbors within a distance ≤ r (r=1,2) from their current block; hide all information beyond this radius.
    - Satisficing: Add a threshold ε to the prompt; only allow the LLM to recommend moving if the target utility ≥ current utility + ε; otherwise, stay directly.

**Note**: Large-scale calls to the LLM will incur costs. Please reasonably control `max_steps` and the call frequency during experiments.

## **Enjoy your own simulation! 🥳**